In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

import matplotlib
matplotlib.style.use('seaborn')

from draftbot import Draft
from itertools import repeat

In [ ]:
draft = Draft()
drafters = draft.draft()

In [ ]:
drafter = drafters[0]

In [ ]:
def history_to_dataframe(history):
    archytypes = list(history[0].keys())
    history_array = np.zeros((len(history), len(archytypes)))
    history_df = pd.DataFrame(history_array, columns=archytypes)
    for idx, preferences in enumerate(history):
        for arch, pref in preferences.items():
            history_df.loc[history_df.index[idx], arch] = pref
    return history_df

In [ ]:
def make_archytype_color_mapping(alpha):
    arch_color_mapping = {
        'W': [0.6, 0.6, 0.6, alpha],
        'U': [0, 0, 1, alpha],
        'B': [0, 0, 0, alpha],
        'R': [1, 0, 0, alpha],
        'G': [0, 0.8, 0, alpha]
    }
    return arch_color_mapping

def plot_alternating_color_line(ax, x, y, colors):
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    segment_boundaries = np.arange(x.min(), x.max() + 1)
    n_bins = len(segment_boundaries) - 1
    cmap = ListedColormap((colors*n_bins)[:n_bins])
    norm = BoundaryNorm(segment_boundaries, cmap.N)
    lc = LineCollection(segments, cmap=cmap, norm=norm)
    lc.set_array(x)
    lc.set_linewidth(2)
    ax.add_collection(lc)
    
def make_archytype_colors(arch, alpha=1):
    arch_color_mapping = make_archytype_color_mapping(alpha)
    return [arch_color_mapping[c] for c in arch]

In [ ]:
def plot_color_identity_dot(ax, x, y, color_identity, alpha=1.0):
    arch_color_mapping = make_archytype_color_mapping(alpha)
    colors = [arch_color_mapping[c] for c in color_identity]
    if len(colors) == 1:
        ax.scatter(x, y, c=[colors[0]], zorder=3)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 4))

history_df = history_to_dataframe(drafter._archytype_preferences_history)
card_colors = [card['colorIdentity'] for card in drafter.cards]

winning_archytype = history_df.max().idxmax()
winning_history = history_df.loc[:, winning_archytype]

colors = make_archytype_colors(winning_archytype, alpha=1.0)
plot_alternating_color_line(ax, winning_history.index.values, winning_history.values, colors)

for arch in set(history_df.columns) - set([winning_archytype]):
    arch_history = history_df.loc[:, arch]
    colors = make_archytype_colors(arch, alpha=0.2)
    plot_alternating_color_line(ax, arch_history.index.values, arch_history.values, colors)

for color_pair, x, y in zip(card_colors, winning_history.index.values, winning_history.values):
    plot_color_identity_dot(ax, x, y, color_pair)    

    ax.set_xlim(0, 36)
ax.set_ylim(0, 34)

In [ ]:
# first define the ratios
r1 = 0.2       # 20%
r2 = r1 + 0.4  # 40%

# define some sizes of the scatter marker
sizes = np.array([60, 80, 120])

# calculate the points of the first pie marker
#
# these are just the origin (0,0) +
# some points on a circle cos,sin
x = [0] + np.cos(np.linspace(0, 2 * np.pi * r1, 10)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r1, 10)).tolist()
xy1 = np.column_stack([x, y])
s1 = np.abs(xy1).max()

x = [0] + np.cos(np.linspace(2 * np.pi * r1, 2 * np.pi * r2, 10)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r1, 2 * np.pi * r2, 10)).tolist()
xy2 = np.column_stack([x, y])
s2 = np.abs(xy2).max()

x = [0] + np.cos(np.linspace(2 * np.pi * r2, 2 * np.pi, 10)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r2, 2 * np.pi, 10)).tolist()
xy3 = np.column_stack([x, y])
s3 = np.abs(xy3).max()

fig, ax = plt.subplots()
ax.scatter(range(3), range(3), marker=(xy1, 0),
           s=s1 ** 2 * sizes, facecolor='blue')
ax.scatter(range(3), range(3), marker=(xy2, 0),
           s=s2 ** 2 * sizes, facecolor='green')
ax.scatter(range(3), range(3), marker=(xy3, 0),
           s=s3 ** 2 * sizes, facecolor='red')